# Temporal Networks v1.0 #

In [1]:
import sys
sys.path.append("/home/genois/Bureau/tempnet")

## Utilities ##
The `utils` module of the library contains utility functions for the temporal networks:
- data reading and writing;
- conversion between representations;
- aggregation of the temporal network;
- various measures on the temporal network;
- quick plot of a network with graph-tools.

In [2]:
import tempnet.utils as tnu

## Data reading & writing ##

All four representations for temporal networks can be used as input and output in text files. The formatting for each is the following:

File type             | Format
--------------------- | ---------------------------
tij.dat               | `t \t i \t j \n`
tijtau.dat            | `t \t i \t j \t tau \n`
snapshot_sequence.dat | `t \t i,j \t ... \t y,z \n`
link_stream.dat       | `i,j \t t_1,tau_1 \t ... \t t_n,tau_n \n`

**Description:**
- The `tij` format is the most common way to store a temporal network: it simply lists each event `(t,i,j)` of duration one unit time `dt`.
- The `tijtau` format is an extension of the previous, where contacts with their duration `tau` are listed instead of events.
- The `snapshot_sequence` format lists all time steps, with for each the list of links active during this time step.
- The `link_stream` format lists all links in the network aggregated over time, with for each the list of contacts.

### `read_tij(filename)` ###
This function reads a file indicated by the `filename` string formatted like a tij.dat file, and returns a `tij()` object.

In [4]:
tij_data = tnu.read_tij("/home/genois/Bureau/tempnet/tij_InVS13.dat")

### `read_tijtau(filename)` ###
This function reads a file indicated by the `filename` string formatted like a tijtau.dat file, and returns a `tijtau()` object.

In [11]:
tijtau_data = tnu.read_tijtau("/home/genois/Bureau/tempnet/tijtau_InVS13.dat")

### `read_snapshot_sequence(filename,t_i=0,t_f=0,dt=0)` ###
This function reads a file indicated by the `filename` string formatted like a snapshot_sequence.dat, and returns a `snapshot_sequence()` object. The initial time `t_i`, the final time `t_f` and the time step duration `dt` can be specified when the function is called. If not, they are automatically extracted from the data to construct the `snapshot_sequence()` object.

In [12]:
seq_data = tnu.read_snapshot_sequence("/home/genois/Bureau/tempnet/timeset_InVS13.dat")

In [3]:
seq_data = tnu.read_snapshot_sequence("/home/genois/Bureau/tempnet/timeset_InVS13.dat",0,60480,1)

### `read_link_stream()` ###
This function reads a file indicated by the `filename` string formatted like a link_stream.dat, and returns a `link_stream()` object.

In [14]:
lks_data = tnu.read_link_timeline("/home/genois/Bureau/tempnet/lks_InVS13.dat")

### `write_tij(filename,tij_data)` ###
This function writes a `tij()` object `tij_data` in a file indicated by the `filename` string, with the proper tij.dat format.

In [15]:
tnu.write_tij("tij.dat",tij_data)

### `write_tijtau(filename,tijtau_data)` ###
This function writes a `tijtau()` object `tijtau_data` in a file indicated by the `filename` string, with the proper tijtau.dat format.

In [16]:
tnu.write_tijtau("tijtau.dat",tijtau_data)

### `write_snapshot_sequence(filename,seq_data)` ###
This function writes a `snapshot_sequence()` object `seq_data` in a file indicated by the `filename` string, with the proper snapshot_sequence.dat format.

In [17]:
tnu.write_snapshot_sequence("timeset.dat",seq_data)

### `write_link_timeline(lks_data)` ###
This function writes a `link_timeline()` object `lks_data` in a file indicated by the `filename` string, with the link_timeline.dat format.

In [18]:
tnu.write_link_timeline("lks.dat",lks_data)

## Data conversion ##
The main conversion is from event-based data to contact-based data, with the assumption that contiguous events are joined to form contacts. Two functions perform this conversion, one based on the simple `tij` and `tijtau` formats, the other based on the high level `snapshot_sequence` and `link_timeline` formats. Along these conversions, we have two natural conversions, namely between `tij` and `snapshot_sequence`, and between `tijtau` and `link_timeline`. The pathways of conversion are thus the following:

**True conversions**
- tij $\leftrightarrow$ tijtau
- snapshot_sequence $\leftrightarrow$ link_timeline

**Natural conversions**
- tij $\leftrightarrow$ snapshot_sequence
- tijtau $\leftrightarrow$ link_timeline

For conversions towards a `snapshot_sequence()`, one can always specify the first and last time stamp, for cases where the first (respectively last) time stamp is different from the first (respectively last) event.

### `tij_to_snapshot_sequence(tij_data,dt,t_i=-1,t_f=0)` ###
This function takes a `tij()` object `tij_data` and returns its conversion to a `snapshot_sequence()` object. Specifying the time step duration `dt` is mandatory. The initial time `t_i` and the final time `t_f` are optional, and are by default set to the first step and last time step + 1 of the `tij()` object. If `t_f` is set at a certain value, the last step will be at time step `t_f-1`.

In [19]:
seq_data = tnu.tij_to_snapshot_sequence(tij_data,1)

In [20]:
seq_data = tnu.tij_to_snapshot_sequence(tij_data,1,0,60480)

### `snapshot_sequence_to_tij(seq_data)` ###
This function takes a `snapshot_sequence()` object `seq_data` and returns its conversion to a `tij()` object.

In [21]:
tij_data = tnu.snapshot_sequence_to_tij(seq_data)

### `tijtau_to_link_timeline(tijtau_data)` ###
This function takes a `tijtau()` object `tijtau_data` and returns its conversion to a `link_timeline()` object.

In [22]:
lks_data = tnu.tijtau_to_link_timeline(tijtau_data)

### `link_timeline_to_tijtau(lks_data)` ###
This function takes a `link_timeline()` object `lks_data` and returns its conversion to a `tijtau()` object.

In [23]:
tijtau_data = tnu.link_timeline_to_tijtau(lks_data)

### `tij_to_tijtau(tij_data,dt,join=True)` ###
This function takes a `tij()` object `tij_data` and returns its conversion to a `tijtau()` object. It necessitates the value of the time step `dt`. The optional argument `join` indicates whether consecutive instant-events should be joined or not to define events with durations. Its default value is `True`.

In [5]:
tijtau_data = tnu.tij_to_tijtau(tij_data,1)

### `tijtau_to_tij(tijtau_data,dt)` ###
This function takes a `tij()` object `tij_data` and returns its conversion to a `tijtau()` object. It necessitates the value of the time step `dt`.

In [25]:
tij_data = tnu.tijtau_to_tij(tijtau_data,1)

### `snapshot_sequence_to_link_timeline(seq_data,dt,join=True)` ###
This function takes a `snapshot_sequence()` object `seq_data` and returns its conversion to a `link_timeline()` object.  It necessitates the value of the time step `dt`. The optional argument `join` indicates whether consecutive instant-events should be joined or not to define events with durations. Its default value is `True`.

In [5]:
lks_data = tnu.snapshot_sequence_to_link_timeline(seq_data,1)

### `link_timeline_to_snapshot_sequence(lks_data,dt,t_i=-1,t_f=0)` ###
This function takes a `link_timeline()` object `lks_data` and returns its conversion to a `snapshot_sequence()` object. Specifying the time step duration `dt` is mandatory. The initial time `t_i` and the final time `t_f` are optional, and are by default set to the first step and list time step + 1 of the `link_timeline()` object. If `t_f` is set at a certain value, the last step will be at time step `t_f-1`.

In [27]:
seq_data = tnu.link_timeline_to_snapshot_sequence(lks_data,1)

In [28]:
seq_data = tnu.link_timeline_to_snapshot_sequence(lks_data,1,0,60480)

## Aggregation ##
In this section we present functions that aggregate a temporal networks along the time. A function is available for each data representation. All functions return a `networkx` `Graph()` object where links have an attribute `w` which gives the total contact duration.

In [4]:
tij_data = tnu.read_tij("/home/genois/Bureau/tempnet/tij_InVS13.dat")
tijtau_data = tnu.read_tijtau("/home/genois/Bureau/tempnet/tijtau_InVS13.dat")
seq_data = tnu.read_snapshot_sequence("/home/genois/Bureau/tempnet/timeset_InVS13.dat")
lks_data = tnu.read_link_timeline("/home/genois/Bureau/tempnet/lks_InVS13.dat")

### `aggregate_tij(tij_data)` ###
This function takes a `tij()` object as input.

In [5]:
G = tnu.aggregate_tij(tij_data)

### `aggregate_tijtau(tijtau_data)` ###
This function takes a `tijtau()` object as input.

In [6]:
G = tnu.aggregate_tijtau(tijtau_data)

### `aggregate_snapshot_sequence(seq_data)` ###
This function takes a `snapshot_sequence()` object as input.

In [7]:
G = tnu.aggregate_snapshot_sequence(seq_data)

### `aggregate_link_timeline(lks_data)` ###
This function takes a `link_timeline()` object as input.

In [8]:
G = tnu.aggregate_link_timeline(lks_data)